# Get Overlapping Width

From the experiment __exp-SphericalAlignment__, which is to test the correctness of camera metrics, we felt strong scepticism on the camera sensor size and focal length values that we get from the JSON file.

This experiment is for measuring AOV(angle of view) and PPD(pixels per degree) by manually estimate overlapping width and height of image frames, thus to ensure the correctness of JSON contents.

Overlapping areas are calculated from matching feature points of adjacent images.

In [1]:
import os
import cv2 as cv
import numpy as np
from libpano import MetaData

Final test data is used for this experiment. We load the JSON file and analysed.

In [2]:
image_id = 'recent-04'
folder = '../images/' + image_id
meta = MetaData.MetaData(folder)
df = meta.grid_data

#### Panorama metrics calculated from the JSON file

In [3]:
print(meta.metrics)

Camera Metrics:
	Focal Length: 4.4589 mm
	Focal Length: 1588.26 px
	Sensor Size: 4.2447996 x 5.6447997 mm
	Pixels per mm: 356.20 x 357.14 px
PanoramaMetrics:
	Frame Count: 15 x 5
	Frame Size: 1512px x 2016px
	Interval Angle: 24.0︒ x 36.0︒
	AoV: 50.9081︒ x 64.6661︒
	AoV: 0.8885 x 1.1286
	PPR: 1701.7168px x 1786.2268px
	Panorama Size: 10692.2019px x 5611.5971px



### Manual Calculation

Overlapping width should be calculated on the middle row of the grid.

In [4]:
center_row = df.row.nunique() // 2
df[df.row == center_row]

,row,col,uri,pitch,roll,yaw
30,2,0,img-r1-167.jpg,1.618314,0.672979,17.196747
31,2,1,img-r1-192.jpg,1.405421,-0.227626,41.420685
32,2,2,img-r1-217.jpg,1.407111,1.317603,66.893280
33,2,3,img-r1-240.jpg,1.669483,1.424618,89.764771
34,2,4,img-r1-264.jpg,1.795640,0.912027,113.928802
35,2,5,img-r1-286.jpg,0.791613,0.535418,136.117828
36,2,6,img-r1-310.jpg,1.689086,1.997290,160.312744
37,2,7,img-r1-335.jpg,1.852506,3.240176,185.285416
38,2,8,img-r1-000.jpg,1.814764,1.945364,209.926773
39,2,9,img-r1-022.jpg,1.616959,2.058648,232.099152


The roll differences between adjacent images are 1.5 degree at most and we think this can be ignored.

Image frames' yaw interval is 24 degree but there are some variations and they should be considered.

In [5]:
uris = df[df.row == center_row].uri.values.tolist()
yaws = df[df.row == center_row].yaw.values.tolist()

#### Function that gets non-overlapping width of two adjacent images

In [6]:
def get_non_overlapping_width(img_name1, img_name2, max_overlap_ratio=0.7):
    # load images
    img1 = cv.imread(img_name1)
    img2 = cv.imread(img_name2)

    # conver into gray scale
    gray1 = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
    gray2 = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)
    
    # find feature points from the gray-scale images
    finder = cv.ORB_create()
    feature1 = cv.detail.computeImageFeatures2(finder, gray1)
    feature2 = cv.detail.computeImageFeatures2(finder, gray2)

    # match key points
    matcher = cv.detail.BestOf2NearestMatcher_create(False, 0.3)
    matches = matcher.apply2([feature1, feature2])
    
    # as the function above returns 4 DMatch objects - 0:0, 0:1, 1:0, 1:1, we should choose correct one 0:1
    real_match = None
    for match in matches:
        if match.src_img_idx == 0 and match.dst_img_idx == 1:
            real_match = match
            break

    # get masked match info
    matches_list = real_match.getMatches()
    matches_mask = real_match.getInliers().ravel().tolist()

    good_matches = []
    for idx, mm in enumerate(matches_mask):
        if mm == 1:
            good_matches.append(matches_list[idx])
    
    # calculate horizontal distance between matched keypoints
    kps1 = feature1.getKeypoints()
    kps2 = feature2.getKeypoints()
    width1 = img1.shape[1]

    non_overlapping_widths = []
    for m in good_matches:
        overlap = kps2[m.trainIdx].pt[0] + width1 - kps1[m.queryIdx].pt[0]
        
        # too large overlap is outliers
        if overlap < width1 * max_overlap_ratio:
            non_overlapping_widths.append(width1 - overlap)

    now = 0
    if len(non_overlapping_widths) > 0:
        now = np.mean(non_overlapping_widths)
            
    # show the matching status        
    draw_params = dict(matchesMask=matches_mask,
                       singlePointColor=None,
                       matchColor=(0, 255, 0),
                       flags=2)
    res = cv.drawMatches(img1, 
                         feature1.getKeypoints(),
                         img2, 
                         feature2.getKeypoints(),
                         matches_list, None, **draw_params)
    cv.putText(res, 
               'Good matches = ' + str(len(non_overlapping_widths)), 
               (50, 100), 
               cv.FONT_HERSHEY_SIMPLEX, 
               1, 
               (0, 0, 255), 
               2, 
               cv.LINE_AA)
    cv.putText(res, 
               'NOW = ' + str(now), 
               (50, 150), 
               cv.FONT_HERSHEY_SIMPLEX, 
               1, 
               (0, 0, 255), 
               2, 
               cv.LINE_AA)
    cv.namedWindow('match', cv.WINDOW_NORMAL)
    cv.resizeWindow('match', 1200, 800)
    cv.imshow('match', res)
    cv.waitKey(0)
    cv.destroyAllWindows()
    
    return now

##### Now, calculate all adjacent now and ppds.

In [7]:
ppds = []
for idx in range(len(uris)-1):
    in1 = os.path.join(folder, uris[idx])
    in2 = os.path.join(folder, uris[idx+1])
    
    # yaw difference between two images
    yaw_diff = yaws[idx+1] - yaws[idx]
    if yaw_diff < 0:
        yaw_diff += 360
        
    # calculate non-overlapping width
    now = get_non_overlapping_width(in1, in2)
    
    # ppd = now / dyaw
    if now != 0:
        ppd = now / yaw_diff
        ppds.append(ppd)
        print('{}~{}: now={}, ppd={}'.format(idx, idx+1, now, ppd))
    else:
        print('{}~{}: no matching points'.format(idx, idx+1, now))
        

0~1: now=772.5613020744877, ppd=31.892473570904436
1~2: no matching points
2~3: now=749.9456617567274, ppd=32.78954043074675
3~4: now=715.8749413123497, ppd=29.625641194032227
4~5: now=692.952143052045, ppd=31.229498168767844
5~6: no matching points
6~7: now=747.5978250676935, ppd=29.936637928568377
7~8: now=503.24998187074567, ppd=20.422981301508695
8~9: now=722.870005607605, ppd=32.60227603918273
9~10: now=851.5272570158306, ppd=33.50880045669968
10~11: now=749.8891052246094, ppd=32.92757328259107
11~12: now=740.2287578830471, ppd=29.97872445876558
12~13: now=677.7042263843974, ppd=29.38757124919135
13~14: no matching points


There are 2 or 3 cases when there can't be found any matches.

As there are many candidates, it's OK for now. But it would be the risk when we stitch images. In this case, we should find out other ways to detect features.

#### Average PPD(Pixels Per Degree)

In [8]:
ppd = np.mean(ppds)
ppd

30.391065280087158

#### The horizontal AOV

In [9]:
1512/ppd

49.75146432233466

## Conclusion

The horizontal angle-of-view **AOV_h** value was calculated from manual estimation. 

The feature matching algorithm depends on random selection of key points so that different results come out at every calculation. The AOV_h changes in the range of 49~50.6 degree while its value was 50.9 from the JSON file.

The maximum deviation is 4% and this showed that we should believe the camera metrics values in despite of our scepticism.

We can try the exp-SphericalAlignment experiment with this 49 degree value.
